In [1]:
import pandas as pd
import glob
import os
import os.path as op
from collections import Counter

from bisect import bisect_left
import numpy as np
from sklearn import neighbors
import pandas as pd
from math import log

from phage_count_table import phage_contig_table, map_clstr_raw
from nb_tools import readfa, swap_cluster_map
from summary import virus_class

In [2]:
clstr = "/mnt/scgc/simon/simonsproject/bats248_annotations/bats248_all_orfs_cdhit_c90.fasta.clstr"
input_fa = "/mnt/scgc/simon/simonsproject/bats248_annotations/bats248_all_orfs.fasta"
phage_hits = "/mnt/scgc/simon/simonsproject/bats248_vs/phage_hits.csv"

#objects needed for generating the phage contig table per SAG in batch:
phage_hits_df = pd.read_csv(phage_hits)
cmap = swap_cluster_map(map_clstr_raw(clstr))

recruits = glob.glob("/mnt/scgc/simon/simonsproject/bats248_vs/diamond/pergenome/*.csv")

to_start =  '''AG−891−A17
AG−892−P18
AG−893−J23
AG−894−C07
AG−895−P08
AG−897−A15  
AG−903−F19
AG−903−I06
AG−904−O13
AG−907−C19
AG−907−I10
AG−908−F15
AG-909-A05
AG−910−E05
AG−912−O18
AG−913−C05
AG−913−C17'''.replace("−","-").split()

recruits = [i for i in recruits if op.basename(i).split("_")[0] in to_start]



In [3]:
ex_recruit = recruits[0]
ex_sag = op.basename(ex_recruit).split("_")[0]
ex_gff = "/mnt/scgc/simon/simonsproject/bats248_annotations/gff/{}.gff".format(ex_sag)
ex_faa = "/mnt/scgc/simon/simonsproject/bats248_annotations/faa/{}.faa".format(ex_sag)

In [8]:
def generate_summary(recruit_tbl, cmap, gff, phage_hits_df):
    rdf = pd.read_csv(ex_recruit).fillna(0)
    mdf = phage_contig_table(cmap, gff, phage_hits_df).reset_index().fillna(0)
    vsdf = pd.merge(mdf, rdf, on='contig', how='outer')
    vsdf['ratio_virus_bacteria'] = vsdf['fr_mg-vir'] / vsdf['fr_mg-bac']
    vsdf['ratio_virus_bacteria'] = [1000 if i == float('inf') else i for i in vsdf['ratio_virus_bacteria']]
    scores = virus_class(vsdf)
    return pd.merge(vsdf, scores, on='contig', how='outer')

In [9]:
for r in recruits:
    ex_recruit = r
    ex_sag = op.basename(ex_recruit).split("_")[0]
    ex_gff = "/mnt/scgc/simon/simonsproject/bats248_annotations/gff/{}.gff".format(ex_sag)
    ex_faa = "/mnt/scgc/simon/simonsproject/bats248_annotations/faa/{}.faa".format(ex_sag)
    out = generate_summary(ex_recruit, cmap, ex_gff, phage_hits_df)
    outfile = "/mnt/scgc/simon/simonsproject/jb_vs_test/{}_new_script.csv".format(ex_sag)
    out.to_csv(outfile)

In [6]:
out

,contig,p1,p2,total_orfs,viral_phage_gene_fraction,viral2_phage_gene_fraction,hit_mg-bac,hit_mg-vir,reads_mg-vir,reads_mg-bac,contig_length,fr_mg-bac,fr_mg-vir,ratio_virus_bacteria,virus_class,virus_prob
0,AG-891-A17_NODE_1,1.0,0.0,166,0.006024,0.0,45869.0,4435.0,5922080.0,8279226.0,147339.0,0.037602,0.005083,0.135173,0.0,1.000000
1,AG-891-A17_NODE_10,0.0,0.0,38,0.000000,0.0,5711.0,743.0,5922080.0,8279226.0,32598.0,0.021161,0.003849,0.181883,0.0,1.000000
2,AG-891-A17_NODE_11,0.0,0.0,20,0.000000,0.0,11935.0,1014.0,5922080.0,8279226.0,24711.0,0.058337,0.006929,0.118777,0.0,1.000000
3,AG-891-A17_NODE_12,0.0,0.0,22,0.000000,0.0,8935.0,489.0,5922080.0,8279226.0,22815.0,0.047303,0.003619,0.076512,0.0,1.000000
4,AG-891-A17_NODE_13,0.0,0.0,29,0.000000,0.0,5901.0,678.0,5922080.0,8279226.0,22000.0,0.032398,0.005204,0.160627,0.0,1.000000
5,AG-891-A17_NODE_14,0.0,0.0,22,0.000000,0.0,4785.0,428.0,5922080.0,8279226.0,20404.0,0.028325,0.003542,0.125048,0.0,1.000000
6,AG-891-A17_NODE_15,0.0,0.0,13,0.000000,0.0,8820.0,589.0,5922080.0,8279226.0,19050.0,0.055922,0.005221,0.093360,0.0,1.000000
7,AG-891-A17_NODE_16,0.0,0.0,14,0.000000,0.0,6814.0,461.0,5922080.0,8279226.0,17004.0,0.048402,0.004578,0.094583,0.0,1.000000
8,AG-891-A17_NODE_17,0.0,0.0,11,0.000000,0.0,212.0,55.0,5922080.0,8279226.0,13103.0,0.001954,0.000709,0.362696,0.0,1.000000
9,AG-891-A17_NODE_18,0.0,0.0,14,0.000000,0.0,2644.0,98.0,5922080.0,8279226.0,12781.0,0.024987,0.001295,0.051818,0.0,1.000000
